In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import demoji
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re, string
import emoji
import pandas as pd

In [4]:
data = pd.read_csv('C:/Users/Pc Bridge/Downloads/Airline-Sentiment-2-w-AA.csv', encoding='ISO-8859-1')
data.head()

_unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
0  681448150    False   finalized                   3      2/25/15 5:24   
1  681448153    False   finalized                   3      2/25/15 1:53   
2  681448156    False   finalized                   3     2/25/15 10:01   
3  681448158    False   finalized                   3      2/25/15 3:05   
4  681448159    False   finalized                   3      2/25/15 5:50   

  airline_sentiment  airline_sentiment:confidence negativereason  \
0           neutral                        1.0000            NaN   
1          positive                        0.3486            NaN   
2           neutral                        0.6837            NaN   
3          negative                        1.0000     Bad Flight   
4          negative                        1.0000     Can't Tell   

   negativereason:confidence         airline airline_sentiment_gold  \
0                        NaN  Virgin America                    NaN   
1                     0.0000  Virgin America                    NaN   
2                        NaN  Virgin America                    NaN   
3                     0.7033  Virgin America                    NaN   
4                     1.0000  Virgin America                    NaN   

         name negativereason_gold  retweet_count  \
0     cairdin                 NaN              0   
1    jnardino                 NaN              0   
2  yvonnalynn                 NaN              0   
3    jnardino                 NaN              0   
4    jnardino                 NaN              0   

                                                text tweet_coord  \
0                @VirginAmerica What @dhepburn said.         NaN   
1  @VirginAmerica plus you've added commercials t...         NaN   
2  @VirginAmerica I didn't today... Must mean I n...         NaN   
3  @VirginAmerica it's really aggressive to blast...         NaN   
4  @VirginAmerica and it's a really big bad thing...         NaN   

   tweet_created      tweet_id tweet_location               user_timezone  
0  2/24/15 11:35  5.703060e+17            NaN  Eastern Time (US & Canada)  
1  2/24/15 11:15  5.703010e+17            NaN  Pacific Time (US & Canada)  
2  2/24/15 11:15  5.703010e+17      Lets Play  Central Time (US & Canada)  
3  2/24/15 11:15  5.703010e+17            NaN  Pacific Time (US & Canada)  
4  2/24/15 11:14  5.703010e+17            NaN  Pacific Time (US & Canada)

In [5]:
data.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _unit_id                      14640 non-null  int64  
 1   _golden                       14640 non-null  bool   
 2   _unit_state                   14640 non-null  object 
 3   _trusted_judgments            14640 non-null  int64  
 4   _last_judgment_at             14584 non-null  object 
 5   airline_sentiment             14640 non-null  object 
 6   airline_sentiment:confidence  14640 non-null  float64
 7   negativereason                9178 non-null   object 
 8   negativereason:confidence     10522 non-null  float64
 9   airline                       14640 non-null  object 
 10  airline_sentiment_gold        40 non-null     object 
 11  name                          14640 non-null  object 
 12  negativereason_gold           32 non-null     object 
 13  r

In [7]:
df = data[['text','airline_sentiment']]
df.head()

text airline_sentiment
0                @VirginAmerica What @dhepburn said.           neutral
1  @VirginAmerica plus you've added commercials t...          positive
2  @VirginAmerica I didn't today... Must mean I n...           neutral
3  @VirginAmerica it's really aggressive to blast...          negative
4  @VirginAmerica and it's a really big bad thing...          negative

In [8]:
df.isnull().sum()

text                 0
airline_sentiment    0
dtype: int64

## Clean  text from emojis

In [9]:
def clean_text_from_emojis(text):
    return demoji.replace(text, '')

## Remove punctuations, links, mentions and \r\n new line characters

In [10]:
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

## Clean hastags '#' Symbol

In [11]:
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2


## Remove multiple spaces

In [12]:
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)


In [13]:
dataset1_texts_new = []
for t in df.text:
    dataset1_texts_new.append(remove_mult_spaces(clean_hashtags(strip_all_entities(clean_text_from_emojis(t)))))
    

In [14]:
df['cleaned_text'] = dataset1_texts_new

df1 = df.reindex(columns=['text', 'cleaned_text', 'airline_sentiment'])

df1 = df1.rename(columns={'airline_sentiment': 'sentiment'})

df1.head()

C:\Users\Pc Bridge\AppData\Local\Temp\ipykernel_17848\2737867872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = dataset1_texts_new


text  \
0                @VirginAmerica What @dhepburn said.   
1  @VirginAmerica plus you've added commercials t...   
2  @VirginAmerica I didn't today... Must mean I n...   
3  @VirginAmerica it's really aggressive to blast...   
4  @VirginAmerica and it's a really big bad thing...   

                                        cleaned_text sentiment  
0                                          what said   neutral  
1  plus youve added commercials to the experience...  positive  
2  i didnt today must mean i need to take another...   neutral  
3  its really aggressive to blast obnoxious enter...  negative  
4            and its a really big bad thing about it  negative

## Randomly select a portion of the data to turn into unlabeled

In [15]:
percentage_to_turn_unlabeled = 0.4 
num_samples_to_turn_unlabeled = int(len(df1) * percentage_to_turn_unlabeled)
samples_to_turn_unlabeled = df1.sample(num_samples_to_turn_unlabeled)

In [16]:
df1.loc[samples_to_turn_unlabeled.index, 'sentiment'] = None  
df1= df1[['cleaned_text','sentiment']]
df1.head()

cleaned_text sentiment
0                                          what said      None
1  plus youve added commercials to the experience...  positive
2  i didnt today must mean i need to take another...      None
3  its really aggressive to blast obnoxious enter...  negative
4            and its a really big bad thing about it      None

In [17]:
vectorizer = TfidfVectorizer()

## Save DataFrame to CSV and Json

In [18]:
import pandas as pd
import pickle

df1.to_csv('df4.csv', index=False)
with open('df4.pkl', 'wb') as f:
    pickle.dump(df, f)

df1.to_json('df4.json', orient='records')